In [69]:
import os,io
from google.cloud import vision
from google.cloud.vision import types
from PIL import Image,ImageDraw,ImageFont
import pandas as pd

In [97]:
#function to draw borders
def drawVertices(image_source,vertices,display_text='',save_to='',save_file_name='annotatedImg.jpg', show=False):
    pillow_img = Image.open(io.BytesIO(image_source))

    draw = ImageDraw.Draw(pillow_img)
    for i in range(len(vertices)-1):
        draw.line(((vertices[i].x,vertices[i].y),(vertices[i+1].x,vertices[i+1].y)),fill='green',width=8)

    draw.line(((vertices[len(vertices)-1].x,vertices[len(vertices)-1].y),(vertices[0].x,vertices[0].y)),fill='green',width=8)

    font = ImageFont.truetype('arial.ttf',16)
    draw.text((vertices[0].x + 10, vertices[0].y), font=font,text=display_text,fill=(255,255,255))

    if show:
        pillow_img.show()
    if save_to!='':
        pillow_img.save(os.path.join(save_to,save_file_name))

#function to create dataframe from responses
def create_df(localized_annotations):
    df = pd.DataFrame(columns=['name','score','locations'])
    for obj in localized_annotations:
        df = df.append(
            dict(
                name=obj.name,
                score=obj.score,
                locations=obj.bounding_poly.normalized_vertices
            ),
            ignore_index=True
        )
    return df

In [7]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'leafy-acumen-268610-a38fd0012f87.json'
client = vision.ImageAnnotatorClient()
client

In [ ]:
#To use images using URIs
img_uri="https://cdnblog.picsart.com/2015/08/178443533002201.jpg"
image = vision.types.Image()
image.source.image_uri = img_uri

In [103]:
#To use images from local system
file_name='instruments1.jpg'
image_folder='images/input_images'
image_path = os.path.join(image_folder,file_name)
with io.open(image_path,'rb') as image_file:
    content = image_file.read() #byte object

image = vision.types.Image(content=content) #vision image object

In [41]:
#client call for text detection
response = client.text_detection(image=image)

In [104]:
#client call for object locatlization
object_loc_response = client.object_localization(image=image)

In [44]:
texts = response.text_annotations
main_text = texts[0].description
vertices = texts[0].bounding_poly.vertices

In [62]:
drawVertices(content,vertices,save_to='images/labelled_images',save_file_name='labelled1.jpg')

In [106]:
df=create_df(object_loc_response.localized_object_annotations)
df
#drawVertices(content,df['locations'][1],save_to='images/labelled_images',save_file_name='labelled2.jpg',display_text='Laptop')

,name,score,locations
0,Drum,0.891770,[x: 0.1760147660970688\ny: 0.36738502979278564...
1,Violin,0.881586,[x: 0.4616948068141937\ny: 0.03589838743209839...
2,Musical instrument,0.868803,[x: 0.7185540199279785\ny: 0.04809379577636719...
3,Musical instrument,0.861224,[x: 0.02480093203485012\ny: 0.0186295025050640...
4,Drum,0.796380,[x: 0.6583530306816101\ny: 0.6760404706001282\...
5,Musical instrument,0.767071,[x: 0.49051544070243835\ny: 0.3896863162517547...
6,Musical instrument,0.718116,[x: 0.0394405759871006\ny: 0.3818153440952301\...
7,Musical instrument,0.677060,[x: 0.032257337123155594\ny: 0.626742124557495...
